In [ ]:
import sys
sys.path.append("..")

import random
import math
import itertools
from copy import deepcopy
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union, Dict

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
import plotly.graph_objects as go
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display, Audio
import torchaudio
from torchaudio.io import StreamReader

from src.datasets import *
from src.algo import GreedyLibrary
from src.util.image import *
from src.util import to_torch_device
from src.patchdb import PatchDB, PatchDBIndex
from src.models.encoder import *
from src.util.audio import *
from src.util.files import *

In [ ]:
filename = "/home/bergi/Music/BR80-backup/ROLAND/LIVEREC/LIVE0033-2023-09-10-poesnek.WAV"
filename = "/home/bergi/Music/Aphex Twin/Acoustica Alarm Will Sound Performs Aphex Twin/01 Cock_Ver 10.mp3"

stream = StreamReader(filename)


In [ ]:
while stream.num_out_streams:
    stream.remove_stream(0)
stream.add_basic_audio_stream(44_000)

In [ ]:
for chunks in stream.stream():
    wave = chunks[0]
    break
wave.shape

In [ ]:
wave._elem.dtype

In [ ]:
Audio(.1 * np.random.randn(20_000), rate=44100, normalize=False)

In [ ]:
Audio(wave._elem.permute(1, 0).numpy(), rate=44000, )

In [ ]:
plot_audio(wave[-256:].permute(1, 0), (200, 400))

In [ ]:
ds = AudioSliceIterableDataset(filename, slice_size=1000, sample_rate=22050, mono=True)
print(next(iter(ds)).shape)
print(next(iter(DataLoader(ds, batch_size=3))).shape)

In [ ]:


            
ds = AudioSliceIterableDataset(filename, 22_050, 1000)
waves = []
for wave in ds:
    waves.append(wave[:, 0])
    if len(waves) > 14:
        break
waves = torch.concat([w.unsqueeze(0) for w in waves][9:]) 
plot_audio(waves, (300, 1000))

In [ ]:
@torch.no_grad()
def plot_audio_LOCAL(
        waves: torch.Tensor, 
        shape: Tuple[int, int],
        colors: Optional[Iterable[Tuple[int, int, int]]] = None,
) -> PIL.Image.Image:
    image = PIL.Image.new("RGB", (shape[-1], shape[-2]))
    draw = PIL.ImageDraw.ImageDraw(image)
    
    draw.line(
        ((0, shape[-2] / 2), (shape[-1], shape[-2] / 2)),
        fill=(128, 128, 128),
        width=1,
    )
    
    if waves.ndim == 1:
        waves = waves.unsqueeze(0)
        
    if colors is None:
        colors = (
            (255, 255, 255),
            (255, 128, 128),
            (128, 255, 128),
            (128, 128, 255),
            (255, 255, 128),
            (255, 128, 255),
            (128, 255, 255),
        )
    else:
        colors = tuple(colors)
    
    for i, wave in enumerate(waves):
        x = torch.linspace(0, shape[-1] - 1, len(wave))
        y = (1. - wave) * (shape[-2] / 2. - 1)
        segments = torch.cat([x.unsqueeze(0), y.unsqueeze(0)]).permute(1, 0)
        segments = tuple(tuple(s) for s in segments)
        draw.line(segments, width=1, fill=colors[i % len(colors)])
    return image

#plot_audio(wave[:1280, 0].view(-1, 128), (128, 256))
plot_audio(-torch.ones(128), (128, 256))

In [ ]:
PIL.ImageDraw.ImageDraw.line?

In [ ]:
from scripts.train_vae_audio import VAEAudioLinear
m = VAEAudioLinear(1000, 100)
m.forward(torch.randn(10, 1, 1000)).shape

In [ ]:
from torchaudio.utils.ffmpeg_utils import get_audio_decoders, get_input_protocols
#get_audio_decoders()
get_input_protocols()